#Load Data Utilities
load_data_utils.ipynb  
This notebook is a collection of utilities used within load data repo primarily when running in colab.

When interactive is True each cell runs a small example, basically a mini unit test.  Main also runs several examples.  When saved as .py interactive gets set to False and these are skipped.

It can be saved as a .py file to create callable functions, see examples below.

Author:  [Lee B. Hinkle](https://userweb.cs.txstate.edu/~lbh31/), [IMICS Lab](https://imics.wp.txstate.edu/), Texas State University, 2022

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

TODO:
* Make option to use local time for log fname in addition to UTC time reported in colab 
* shapes function does not seem to be callable outside of Jupyter notebook, need to replace with tabulate type method.



In [1]:
# Interactive mode is intended for debugging as .ipynb
# These are global variables see this reference:
# https://docs.python.org/3/faq/programming.html#what-are-the-rules-for-local-and-global-variables-in-python
interactive = True
log_info = "Example log file generated using load_data_utils.ipynb.\n"

In [ ]:
# This cell runs automatically when .py version called, skip to debug or run .pynb version
interactive = False

In [2]:
if interactive:
    print("Interactive mode on.")
    print("---- Contents of log_info ----")
    print(log_info)

Interactive mode on.
---- Contents of log_info ----
Example log file generated using load_data_utils.ipynb.



In [3]:
from requests import get # for what is my name
import os # for get_env_info
import time # for timestamp in get_log_fname
import numpy as np # for get_shapes
import urllib.request # to get files from web w/o !wget
import csv # to read csv files

# Start of the utility functions

In [7]:
import sys # putting this here, not sure if function is worth it!
# ref: https://stackoverflow.com/questions/53581278/test-if-notebook-is-running-on-google-colab
def running_in_colab():
    """Looks for string 'google.colab' in sys.modules.  Returns True if found."""
    return ('google.colab' in sys.modules)
if interactive:
    print(running_in_colab())

True


In [8]:
def get_env_info():
    """returns the CPU and GPU info, only tested in colab.  Empty GPU indicates
    running in colab instance without GPU acceleration."""
    #cpu_model = !grep 'model name' /proc/cpuinfo
    #env_info = 'CPU1: ' + cpu_model[0] + '\n'
    #env_info += 'CPU2: ' + cpu_model[1] + '\n'
    # the code above uses a magic which runs only in iPython, does not work
    # when imported to other notebooks.  Acknowledgement to 
    # https://janakiev.com/blog/python-shell-commands/ for this alternative
    if os.path.isfile('/proc/cpuinfo'):
        stream = os.popen('grep "model name" /proc/cpuinfo')
        env_info = stream.read()
    else:
        env_info = "/proc/cpuinfo not found, cannot determine CPUs\n"

    if running_in_colab():
        stream2 = os.popen('nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv')
        gpu_info = stream2.read()
        if 'failed' in gpu_info:
            env_info += 'GPU: NVIDIA-SMI failed, no GPU found'
        else:
            env_info += 'GPU: ' + gpu_info
    else:
        env_info += "Not running in colab, GPU info unknown"
    return env_info
if interactive:
    print("get_env_info returned\n" + get_env_info())

get_env_info returned
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
GPU: NVIDIA-SMI failed, no GPU found


In [9]:
def get_log_ffname(
    log_file_dir = '.',
    base_fname = "unnamed"):
    """checks that log_file_dir exists and returns unique full filename
    which is concatenation of log_file_dir, base_name, and current UTC time"""
    if (not os.path.isdir(log_file_dir)):
        print("WARNING: " + log_file_dir + " directory does not exist.")
    timestamp = time.strftime('%b-%d-%Y_%H%M', time.localtime()) #UTC time
    log_fname = base_fname +'_' + timestamp + '.txt'
    log_ffname = os.path.join(log_file_dir, log_fname)
    # I use fname for name only, and ffname for full path and name
    if (os.path.isfile(log_ffname)):
        print("WARNING:",log_ffname,"already exist.")
    return log_ffname
if interactive:
    print("Returned full filename",get_log_ffname(base_fname = "interactive_test"))

Returned full filename ./interactive_test_Dec-13-2022_2220.txt


In [10]:
def csv_to_latex(csv_ffname):
    """A pretty rudimentary converter from csv to LaTeX to save some typing.
    input is full filename of a .csv file, returns a long string of LaTeX.
    This is mostly because I didn't want to throw away code I wrote for a
    specific paper.  There are probably much more capable libraries!"""
    latex_str = '%Table generated from ' + csv_ffname + ' by csv_to_latex\n'
    def escape(line):
        """Escapes special LaTeX characters by prefixing them with backslash, 
        credit: https://github.com/narimiran/tably/blob/master/tably.py"""
        for char in '#$%&_}{':
            line = [column.replace(char, '\\'+char) for column in line]
        line = ''.join(line) # added so that a string not list of strings is returned
        return line
    # Note: the frequent backslashes in LaTeX must be escaped in Python strings
    #print('%Table generated from',csv_ffname,'by csv_to_latex')
    col_sep = '&' # for import as LaTeX table
    end_line = ' \\\\' # end of table row with underline for latex
    hline = '\\hline'
    # Have to read the first row to figure out number of columns required.
    with open(csv_ffname, newline='') as f:
        reader = csv.reader(f)
        for row in reader:
            if(reader.line_num==1):  # this is the first row (header)
                num_columns = len(row)
                cols = ''
                for i in (range(num_columns)):
                    cols += 'c' # to make the {ccc} thing in LaTeX
                table_begin = '\\begin{table}[h]\n' + \
                    '  \\caption{insert caption here}\n' + \
                    '\\label{tab:insert_label}\n' + \
                    '\\begin{tabular}{' + cols + '}\n' + \
                    '\\hline\n'
                latex_str += table_begin
            if(len(row) != num_columns):
                print("Warning: row",reader.line_num,"has different number of elements than header row")
            my_line = ''
            for item in row:
                my_line += item + col_sep
            my_line = my_line[:-1] # get rid of last col_sep
            my_line += end_line
            latex_str += my_line + '\n'
    table_end = '\\hline\n' + '\\end{tabular}\n' + '\\end{table}'
    latex_str += table_end + '\n'
    return latex_str
if interactive:
    import csv
    # Setup a few test files ref: https://docs.python.org/3/library/csv.html
    # also https://levelup.gitconnected.com/building-csv-strings-in-python-32934aed5a9e
    data = [['Name', 'Age', 'Salary'], ['Bob', '45', '75000'], ['Andrew', '34', '79000']]
    with open('csv_to_latex_test.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(data)
    returned_str = csv_to_latex(csv_ffname='/content/csv_to_latex_test.csv')
    print(returned_str)

%Table generated from /content/csv_to_latex_test.csv by csv_to_latex
\begin{table}[h]
  \caption{insert caption here}
\label{tab:insert_label}
\begin{tabular}{ccc}
\hline
Name&Age&Salary \\
Bob&45&75000 \\
Andrew&34&79000 \\
\hline
\end{tabular}
\end{table}



In [11]:
def tabulate_numpy_arrays(dict_name_npy):
    """Returns a string of tabulated data for numpy arrays passed as dictionary.
    args: dictionary format of {"npy_array_name":npy_array,...}.
    This one is pretty narrowly tested, mostly for trainX, testy etc."""
    # it seems silly to pass the variable name as a string, but I haven't found
    # a method that is portable/callable to get the variable name.
    from tabulate import tabulate
    headers = ("array","shape", "data type")
    meta_data = []
    for i in dict_name_npy :
        meta_data.append ((i,str(dict_name_npy[i].shape),str(dict_name_npy[i].dtype)))
    return(tabulate(meta_data, headers=headers))
if interactive:
    randX = np.random.rand(42,10,3)
    randy = np.random.rand(42,6)
    my_dict = {"randX": randX, "randy":randy}
    print(tabulate_numpy_arrays(my_dict))

array    shape        data type
-------  -----------  -----------
randX    (42, 10, 3)  float64
randy    (42, 6)      float64


In [6]:
def channel_powerset(ch_list):
    """creates a powerset of all channel combos except the empty set.
    param: ch_list - list of channel names
    returns: a list of lists of all channel name combinations"""
    # credit: https://towardsdatascience.com/the-subsets-powerset-of-a-set-in-python-3-18e06bd85678
    # powerset is also available in more-itertools
    from itertools import chain, combinations
    return chain.from_iterable(combinations(ch_list, r+1) for r in range(len(ch_list)))
    # note the modified indexing above - this is to eliminate the 1st empty set
if (interactive):
    sth = ['A','B','C'] 
    for x in channel_powerset(sth):
        print(list(x)) #print(', '.join(list(x))) # to print w/o brackets

['A']
['B']
['C']
['A', 'B']
['A', 'C']
['B', 'C']
['A', 'B', 'C']


# Main:  Examples of basic setup and text logging


In [12]:
if __name__ == "__main__":
    log_info += "Running load_data_utils __main__ (example and basic tests)\n"
    log_info += "running_in_colab() returned " + str(running_in_colab())+"\n"
    log_info += "get_env_info returned\n" + get_env_info() + "\n"
    log_info += "get_log_ffname_returned: " + get_log_ffname(base_fname = "main_test")
    print(log_info) # monitor the info to be written when debugging

    # This is the key part - gets a unique file name and saves results to file.
    # If you run this cell repeatedly each run appends the info.
    save_log = True
    log_ffname = get_log_ffname(
        log_file_dir = '.',
        base_fname = 'just_a_test')
    # This version appends, so for each pass make a new log_info file and
    # run at end of each pass so each one is recorded in case of crash.
    # alternative is to run both and generate seperate files for each.
    if save_log:
        print("Writing log_info to " + log_ffname)
        with open(log_ffname, "a") as file_object:
            file_object.write(log_info)
    print ('\n------ Example of .csv converted to LaTeX------')
    urllib.request.urlretrieve('https://people.sc.fsu.edu/~jburkardt/data/csv/ford_escort.csv', filename='ford_escort.csv')
    print(csv_to_latex(csv_ffname='ford_escort.csv'))
    print ('\n------ Example of tabulated numpy array info ------')
    randX = np.random.rand(42,10,3)
    randy = np.random.rand(42,6)
    print(tabulate_numpy_arrays({"randX": randX, "randy":randy}))
    print ('\n------ Example of a powerset (all channel combos) ------')
    for x in channel_powerset(['Ch1','Ch2','Ch3']):
        print(list(x))

Example log file generated using load_data_utils.ipynb.
Running load_data_utils __main__ (example and basic tests)
running_in_colab() returned True
get_env_info returned
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
GPU: NVIDIA-SMI failed, no GPU found
get_log_ffname_returned: ./main_test_Dec-13-2022_2221.txt
Writing log_info to ./just_a_test_Dec-13-2022_2221.txt

------ Example of .csv converted to LaTeX------
%Table generated from ford_escort.csv by csv_to_latex
\begin{table}[h]
  \caption{insert caption here}
\label{tab:insert_label}
\begin{tabular}{ccc}
\hline
Year& "Mileage (thousands)"&  "Price" \\
1998&  27&    9991  \\
1997&  17&    9925  \\
1998&  28&   10491  \\
1998&   5&   10990  \\
1997&  38&    9493  \\
1997&  36&    9991  \\
1997&  24&   10490  \\
1997&  37&    9491  \\
1997&  38&    9491  \\
1997&  30&    9990  \\
1997&  38&    9491  \\
1997&  25&    9990  \\
1997&  39&    9990  \\
1997&  22&    9390  \\
1997&  24&    9990  \\
1

# Example calling code - paste this into another notebook to use public version

In [ ]:
# def get_load_data_utils():
#     """checks for local file, if none downloads from IMICS repository.
#     Assumes a global my_dir has been defined (default is my_dir = ".")
#     :return: nothing"""
#     fname = 'load_data_utils.py'
#     ffname = os.path.join(my_dir,fname)
#     if (os.path.exists(ffname)):
#         if verbose:
#             print ("Local load_data_utils.py found, skipping download")
#     else:
#         print("Downloading",fname, "from IMICS git repo")
#         urllib.request.urlretrieve("https://raw.githubusercontent.com/imics-lab/load_data_time_series/main/load_data_utils.py", filename=fname)
# if interactive:
#     get_load_data_utils()

In [ ]:
# from load_data_utils import get_env_info
# from load_data_utils import get_log_ffname

# print('My env_info: \n', get_env_info())
# print('A new log_ffname: ', get_log_ffname(log_file_dir = '/content/drive/My Drive/temp'))

# The code gutter - some with issues some just examples to have around
Examples to split and append ffname to save figures, pdf, etc.
Sources from older code so not fully tested in this context. 

In [ ]:
# This works fine in colab to find the .ipynb name, but fails as callable function.
# def what_is_my_name():
#     """returns the name of the running ipynb file if running in colab"""
#     #code is readily available on web - not original
#     if running_in_colab():
#         my_name = get('http://172.28.0.2:9000/api/sessions').json()[0]['name']
#     else:
#         # seems a bit complicated to get filename in any environment
#         # and hard coding the name here will make this not portable
#         # this should elminate the failure in non-colab instances though
#         my_name = 'not running in colab, filename unknown'
#     return my_name
# if interactive:
#     print("what_is_my_name returned " + what_is_my_name())

In [ ]:
#Helper function since frequently checking and logging shapes
#credit https://stackoverflow.com/users/4944093/george-petrov for name method
#this method has name issues when being called.
#this may not work as .py and also should be replaced with tabulate version.
# def namestr(obj, namespace):
#     return [name for name in namespace if namespace[name] is obj]
# def get_shapes(np_arr_list):
#     """Returns text, each line is shape and dtype for numpy array in list
#        example: print(get_shapes([X_train, X_test, y_train, y_test]))"""
#     shapes = ""
#     for i in np_arr_list:
#         my_name = namestr(i,globals())
#         shapes += (my_name[0] + " shape is " + str(i.shape) \
#             + " data type is " + str(i.dtype) + "\n")
#     return shapes
# if interactive:
#     # Just some empty arrays using a typical shape for a better example
#     x_train = np.zeros((15, 300, 3)) # 15 samples with 300 steps of 3 signals
#     y_train = np.zeros((15))
#     x_test = np.zeros((5, 300, 3)) # 15 samples with 300 steps of 3 signals
#     y_test = np.zeros((5))
#     print("Get shapes returned\n" + get_shapes([x_train, y_train, x_test, y_test]))

In [ ]:
# #save the last plot (a current active fig) as pdf
# full_fig_fname = log_full_fname.split('.')[0] + '_loss.pdf'
# print("saving loss plot as",full_fig_fname)
# fig.savefig(full_fig_fname,format='pdf')

# #save the confusion matrix figure as pdf
# full_fig_fname = log_full_fname.split('.')[0] + '_cm.pdf'
# print("saving cm plot as",full_fig_fname)
# fig.savefig(full_fig_fname,format='pdf')

# #save the model summary
# model = keras.models.load_model('my_1D_CNN_model')
# my_model = model.summary()
# full_model_fname = log_full_fname.split('.')[0] + '_model.txt'
# with open(full_model_fname, "a") as file_object:
#     model.summary(print_fn=lambda x: file_object.write(x + '\n'))
#     #acknowledgement https://stackoverflow.com/users/14951382/sparklingdew

# #plot the model using keras function
# #ref https://keras.io/api/utils/model_plotting_utils/
# full_mplot_fname = log_full_fname.split('.')[0] + '_model.png'
# tf.keras.utils.plot_model(model, to_file=full_mplot_fname, show_shapes=True)

# #save the predicted activity plot as pdf
# full_fig_fname = log_full_fname.split('.')[0] + '_pred.pdf'
# print("saving prediction plot as",full_fig_fname)
# fig.savefig(full_fig_fname,format='pdf')

# #write a sample text file separate from main log file
# full_txt_fname = log_full_fname.split('.')[0] + '_labels.txt'
# text_file = open(full_txt_fname, "w")
# n = text_file.write(my_labels)
# text_file.close()